In [2]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dschr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import pandas as pd

In [5]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [6]:
# You have to adjust the 
df_arguments = pd.read_csv("../KPA_2021_shared_task/kpm_data/arguments_train.csv")
df_keypoints = pd.read_csv("../KPA_2021_shared_task/kpm_data/key_points_train.csv")
df_labels = pd.read_csv("../KPA_2021_shared_task/kpm_data/labels_train.csv")

In [7]:
df_arguments.head()

,arg_id,argument,topic,stance
0,arg_0_0,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,-1
1,arg_0_1,A patient should be able to decide when they h...,Assisted suicide should be a criminal offence,-1
2,arg_0_2,a person has the right to end their suffering ...,Assisted suicide should be a criminal offence,-1
3,arg_0_3,a person should have the dignity to choose how...,Assisted suicide should be a criminal offence,-1
4,arg_0_4,a person should have the right to be able to c...,Assisted suicide should be a criminal offence,-1


In [6]:
key_points = df_keypoints['key_point']

In [7]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [8]:
key_point_embeddings = sbert_model.encode(key_points)

In [9]:
query_arg = df_arguments["argument"][]1
query_arg_id = df_arguments["arg_id"][1]

In [11]:
print(query_arg_id)

arg_0_1


In [13]:
query_vec = sbert_model.encode([query_arg])[0]

In [16]:
l_arg = []
l_sim = []
for kp in key_points:
  sim = cosine(query_vec, sbert_model.encode([kp])[0])
  l_arg.append(kp)
  l_sim.append(sim)
  # print("Sentence = ", sent, "; similarity = ", sim)
    
result = pd.DataFrame({"key_point": l_arg, "similarity": l_sim})

In [17]:
result.head()

,key_point,similarity
0,Assisted suicide gives dignity to the person t...,0.518209
1,Assisted suicide reduces suffering,0.526782
2,People should have the freedom to choose to en...,0.633044
3,The terminally ill would benefit from assisted...,0.557968
4,Assisted suicide allows people to solicit some...,0.462178


In [20]:
most_similar_kp = result.loc[result['similarity'].idxmax()]
most_similar_kp_id = df_keypoints[df_keypoints["key_point"]==most_similar_kp["key_point"]]["key_point_id"].iloc[0]
print(most_similar_kp_id)

kp_13_4


In [22]:
correct_kp = df_labels[df_labels["arg_id"]==query_arg_id]
correct_kp = correct_kp[correct_kp["label"]==1]["key_point_id"].iloc[0]
print(correct_kp)

kp_0_2


In [25]:
print("Query: ", query_arg)
print("Key_point is: ", most_similar_kp["key_point"])
if correct_kp == most_similar_kp_id:
    print("Matching: Correct")
else:
    print("Matching: False")

Query:  A patient should be able to decide when they have had enough "care".
Key_point is:  The parents should decide for their child
Matching: False
